# Optical flow

In [ ]:
# Install libraries
!pip install -r requirements.txt

In [1]:
# Import libraries
import cv2

print("Installed opencv version", cv2.__version__)

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

print("Installed numpy version", np.__version__)

Installed opencv version 4.10.0
Installed numpy version 1.26.4


In [3]:
import numpy as np
import os
import cv2
import time
from IPython.core.display import display, HTML

display(HTML("<style>"
			 + "#notebook { padding-top:0px !important; } "
			 + ".container { width:100% !important; } "
			 + ".end_space { min-height:0px !important; } "
			 + "</style>"))

# params for ShiTomasi corner detection
feature_params = dict(maxCorners=100,
					  qualityLevel=0.3,
					  minDistance=7,
					  blockSize=7)

# Parameters for lucas kanade optical flow
lk_params = dict(winSize=(15, 15),
				 maxLevel=2,
				 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

vidPath = "data/test.avi"

/var/folders/3m/522x29411p7054b0gby5t_km0000gn/T/ipykernel_13842/3880201517.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


In [6]:
cap = cv2.VideoCapture(vidPath)

# Create some random colors
color = np.random.randint(0, 255, (100, 3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask=None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
etime = 0
while 1:
	ret, frame = cap.read()
	if frame is None:
		break
	frame_gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)

	# calculate optical flow
	t = time.perf_counter()
	p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
	etime += (time.perf_counter() - t)

	# Select good points
	good_new = p1[st == 1]
	good_old = p0[st == 1]

	# draw the tracks
	for i, (new, old) in enumerate(zip(good_new, good_old)):
		a, b = new.astype('int').ravel()
		c, d = old.astype('int').ravel()
		mask = cv2.line(mask, (a, b), (c, d), color[i].tolist(), 2)
		frame = cv2.circle(frame, (a, b), 5, color[i].tolist(), -1)
	img = cv2.add(frame, mask)

	cv2.imshow('frame', img)
	k = cv2.waitKey(30) & 0xff
	if k == 27:
		break

	# Now update the previous frame and previous points
	old_gray = frame_gray.copy()
	p0 = good_new.reshape(-1, 1, 2)

cv2.destroyAllWindows()
cap.release()
